In [1]:
import pandas as pd
import math

In [2]:
# import the files as dataframes
# Customize this based on where the data is coming from

location_list = pd.read_csv('LMFull.csv')
tweet_list = pd.read_csv('TweetTest_Unmatched.csv')


In [3]:
print "# of tweets: " + str(len(tweet_list))
print "# of landmarks: " + str(len(location_list))


# of tweets: 3163
# of landmarks: 1555


In [4]:
tweet_list[:10]

,id,created,text,lat,long,category
0,7.088640e+17,3/13/2016 3:57,06:56 @KenyaRedCross bad accident on Waiyaki W...,0,0,NONE
1,7.088560e+17,3/13/2016 3:24,06:23 ma3route there is an accident along msa ...,0,0,NONE
2,7.088020e+17,3/12/2016 23:48,"02:47 Accident, Langata road. Pedestrian kille...",0,0,NONE
3,7.087740e+17,3/12/2016 21:56,00:55 are people blowing on KIAMBU ROAD? via @...,0,0,NONE
4,7.087630e+17,3/12/2016 21:14,00:14 someone knocked down and died on the spo...,0,0,NONE
5,7.087610e+17,3/12/2016 21:07,00:07 accident at the Junction towards Carnivo...,0,0,NONE
6,7.087590e+17,3/12/2016 20:58,23:57 @IkeOjuok The inhuman acts by police are...,0,0,NONE
7,7.087540e+17,3/12/2016 20:37,23:37 SHARE and remind a family member or a fr...,0,0,NONE
8,7.087480e+17,3/12/2016 20:16,23:15 FYA https://t.co/zf57m5lq2P via @IkeOjuok,0,0,NONE
9,7.087480e+17,3/12/2016 20:15,23:15 The wreckage of the #SubuigaAccident ht...,0,0,NONE


In [5]:
location_list[:10]

,Landmark,Road,Landmark_road,Lat,Lon
0,87,Waiyaki Way,87 Waiyaki Way,-1.257841,36.700929
1,1824,NaN,1824,-1.322725,36.803513
2,1st Parklands Junction,Masari Rd,1st Parklands Junction Masari Rd,-1.263199,36.815862
3,1st Parklands Junction,Masari Road,1st Parklands Junction Masari Road,-1.263199,36.815862
4,87 stage,Waiyaki Way,87 stage Waiyaki Way,-1.257841,36.700929
5,abc,Waiyaki Way,abc Waiyaki Way,-1.259264,36.776663
6,ABC Place,Waiyaki Way,ABC Place Waiyaki Way,-1.259264,36.776663
7,Adams Arcade,Ngong Rd,Adams Arcade Ngong Rd,-1.300039,36.780856
8,Adams Arcade,Ngong Road,Adams Arcade Ngong Road,-1.300039,36.780856
9,adams roundabout,NaN,adams roundabout,-1.300065,36.779716


In [8]:
# Find out how many tweets are missing geo info

def sum_locations(df):
    i=0
    j=0
    for row in df.itertuples():
        if row[4]==0 or row[5]==0:
            i+=1
        else:
            j+=1
    return str(j)+" have lat-long data, "+str(i)+" don't"


In [9]:
print sum_locations(tweet_list)

0 have lat-long data, 3163 don't


In [10]:
# This function goes through each entry in location_list and looks for it in the passed string
# When it finds a match, it returns the associated lat-long as a dict
# It ignores case. If there are multiple matches only the last one will be returned

def check_landmark(test_string):
    result = {'lat':'NONE','lon':'NONE'}
    for landmark in location_list.itertuples():
        if str(landmark[1]).lower() in test_string.lower():
            result = {'lat':landmark[4],'lon':landmark[5]}
    return result

In [11]:
print check_landmark('abc place')
print check_landmark('ABC place')
print check_landmark('ABC place and Adams Arcade')
print check_landmark('fake name')

{'lat': '-1.259264', 'lon': 36.776662999999999}
{'lat': '-1.259264', 'lon': 36.776662999999999}
{'lat': '-1.300039', 'lon': 36.780856}
{'lat': 'NONE', 'lon': 'NONE'}


In [12]:
# Now use the function on every tweet that is missing geo data
# Any missing lat-long values are replaced by matched ones from location_list
# 'NONE' is filled in if no location match was found
def fill_in_locations(df, lat_col=3, lon_col=4):
    for row in df.itertuples():
        idx = row[0]
        location = check_landmark(str(row[3]))
        if (row[4]==0 or row[5]==0):
            df.iloc[idx,lat_col] = location['lat']
            df.iloc[idx,lon_col] = location['lon']

In [13]:
fill_in_locations(tweet_list)

In [14]:
# Check to see how many we fixed

m=0
n=0
for row in tweet_list.itertuples():
    if row[4]=='NONE' or row[5]=='NONE':
        m+=1
    else:
        n+=1

print str(n)+" have lat-long data, "+str(m)+" don't"

1389 have lat-long data, 1774 don't


In [15]:
tweet_list[:50]

,id,created,text,lat,long,category
0,7.088640e+17,3/13/2016 3:57,06:56 @KenyaRedCross bad accident on Waiyaki W...,-1.262821,36.7648,NONE
1,7.088560e+17,3/13/2016 3:24,06:23 ma3route there is an accident along msa ...,-1.180448,36.9391,NONE
2,7.088020e+17,3/12/2016 23:48,"02:47 Accident, Langata road. Pedestrian kille...",-1.249806,36.8468,NONE
3,7.087740e+17,3/12/2016 21:56,00:55 are people blowing on KIAMBU ROAD? via @...,NONE,NONE,NONE
4,7.087630e+17,3/12/2016 21:14,00:14 someone knocked down and died on the spo...,NONE,NONE,NONE
5,7.087610e+17,3/12/2016 21:07,00:07 accident at the Junction towards Carnivo...,-1.298857,36.7621,NONE
6,7.087590e+17,3/12/2016 20:58,23:57 @IkeOjuok The inhuman acts by police are...,NONE,NONE,NONE
7,7.087540e+17,3/12/2016 20:37,23:37 SHARE and remind a family member or a fr...,NONE,NONE,NONE
8,7.087480e+17,3/12/2016 20:16,23:15 FYA https://t.co/zf57m5lq2P via @IkeOjuok,NONE,NONE,NONE
9,7.087480e+17,3/12/2016 20:15,23:15 The wreckage of the #SubuigaAccident ht...,-1.249806,36.8468,NONE


In [ ]:
# Add code here to export the dataframe to a file or database of your choice